In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
from google.colab import drive
drive.flush_and_unmount()
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [2]:
file = 'gdrive/My Drive/DATLAS/ChoquesYSiniestros_2020.csv'
df = pd.read_csv(file, encoding='latin-1')

In [3]:
df.head()

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE
0,D00001,22.768507,-102.567896,98000,*EP 2 Boulevard Adolfo López Mateos,CERRADA DE CASA BLANCA,COLISION Y/O VUELCO,Camión,BLANCO,2016,Bajo,NaN,2018.0,1.0,11.0,JUEVES,15.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,11/1/2018
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018.0,1.0,25.0,JUEVES,10.0,NUEVO LEÓN,JUAREZ,25/01/2018
2,D00003,20.679917,-103.368657,44600,AV MEXICO,2 RIOS,COLISION Y/O VUELCO,Auto,BLANCO,2005,Sin daño,Trasero,2018.0,1.0,15.0,LUNES,9.0,NUEVO LEÓN,GUADALUPE,15/01/2018
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018.0,1.0,19.0,VIERNES,9.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,19/01/2018
4,D00005,19.266303,-99.331791,\N,UBERP3,Cuajimalpa,HUELGAS Y ALBOROTOS,Auto,NEGRO,2016,Medio,NaN,2018.0,1.0,3.0,MIERCOLES,16.0,NUEVO LEÓN,MONTERREY,3/1/2018




*   Asignar geohashes a las latitudes
*   Asignar tipo de camino
*   Asignar si es intersección o no



In [4]:
pip install pygeohash

In [5]:
import pygeohash as pgh

In [6]:
def clean_LAT_LON(data):
    # Create a copy of the dataframe
    df = data.copy()
    # Change all the latitudes and latitudes that does not correspond of NL to NaN
    df['LAT'][(df['LAT'] > 26.0578) | (df['LAT'] < 25.3345)] = np.nan
    df['LONG'][(df['LONG'] < -100.7117) | (df['LONG'] > -99.9234)] = np.nan
    # Quitar aquellas columnas con NaN values
    df = df.dropna(how='any', subset=['LAT','LONG'])

    return df

df = clean_LAT_LON(df)

print(df['LAT'].isna().sum())
print(df['LONG'].isna().sum())

0
0


In [7]:
df['gh4'] = ''
df['gh5'] = ''
df['gh6'] = ''
df['gh7'] = ''
df['gh8'] = ''

for index, row in df.iterrows():
  lat = row['LAT']
  lon = row['LONG']

  ghash = pgh.encode(lat, lon, precision=8)

  df['gh4'][index] = ghash[:-4]
  df['gh5'][index] = ghash[:-3]
  df['gh6'][index] = ghash[:-2]
  df['gh7'][index] = ghash[:-1]
  df['gh8'][index] = ghash

In [8]:
df.head()

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE,gh4,gh5,gh6,gh7,gh8
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018.0,1.0,25.0,JUEVES,10.0,NUEVO LEÓN,JUAREZ,25/01/2018,9u8c,9u8cc,9u8cck,9u8cckb,9u8cckbn
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018.0,1.0,19.0,VIERNES,9.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,19/01/2018,9u8d,9u8dq,9u8dqc,9u8dqcb,9u8dqcb4
6,D00007,25.660916,-100.282504,64820,Avenida Revolución,Valle del Huajuco,COLISION Y/O VUELCO,Auto,PLATA,2006,NaN,NaN,2018.0,1.0,22.0,LUNES,12.0,NUEVO LEÓN,MONTERREY,22/01/2018,9u89,9u89y,9u89yp,9u89yp2,9u89yp2t
7,D00008,25.723590,-100.198780,67117,Boulevard Acapulco,REAL DE SAN MIGUEL,COLISION Y/O VUELCO,Auto,BLANCO,2000,Sin daño,Frontal,2018.0,1.0,29.0,LUNES,17.0,NUEVO LEÓN,GUADALUPE,29/01/2018,9u8d,9u8dr,9u8drf,9u8drfv,9u8drfv6
8,D00009,25.708889,-100.246876,66480,PUERTO MARQUEZ 805,NUEVO MUNDO,COLISION Y/O VUELCO,Auto,AMARILLO,2011,Bajo,Frontal,2018.0,2.0,26.0,LUNES,2.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,26/02/2018,9u8d,9u8dq,9u8dqb,9u8dqb4,9u8dqb4m


In [9]:
import requests

In [10]:
pip install overpy

In [11]:
PATH_TO_SAVE = "gdrive/My Drive/DATLAS/ChoquesYSiniestros_2020GEOHASHED.csv'"

df.to_csv(PATH_TO_SAVE)

In [12]:
import xml.etree.ElementTree as etree

In [13]:
XML_PATH = "gdrive/My Drive/DATLAS/map.xml"

In [14]:
tree = etree.parse(XML_PATH)

In [15]:
root = tree.getroot()

In [16]:
root.tag

'osm'

In [17]:
root.attrib

{'generator': 'Overpass API 0.7.56.7 b85c4387', 'version': '0.6'}

In [18]:
ctr = 0
for child in root:
  if(child.tag == 'node'):
    lat = child.attrib.get('lat')
    lon = child.attrib.get('lon')
    ghash = pgh.encode(float(lat), float(lon), precision=7)
    child.set('ghash', str(ghash))
    ctr += 1

print(ctr)

978105


In [19]:
node_dict = {}

for child in root:
  if child.tag == 'way':
    highwaytype = ''
    for tag in child:
      if tag.tag == 'tag':
        if tag.attrib.get('k') == 'highway':
          highwaytype = tag.attrib.get('v')
    if highwaytype != '':
      for node in child:
        if node.tag == 'nd':
          #Aqui buscar ref en nodos:
          nodeid = node.attrib.get('ref')
          node_dict[nodeid] = highwaytype


In [20]:
len(node_dict)

549448

In [21]:
ctr = 0
for child in root:
  if(child.tag == 'node'):
    id = child.attrib.get('id')
    try:
      htype = node_dict[id]
    except:
      htype = 'NOT A WAY'
    child.set('way_type', htype)
    ctr += 1

print(ctr)

978105


In [22]:
ghash_way_dict = {}

for child in root:
  if child.tag == 'node':
    if child.attrib.get('way_type') != 'NOT A WAY':
      ghash = child.attrib.get('ghash')
      wtype = child.attrib.get('way_type')
      if ghash in ghash_way_dict:
        ghash_way_dict[ghash].append(wtype)
      else:
        ghash_way_dict[ghash] = [wtype]

In [23]:
len(ghash_way_dict)

73457

In [24]:
df['way_type'] = ''
fail_count = 0

for index, row in df.iterrows():
  geohash = row['gh7']
  try:
    df['way_type'][index] = ghash_way_dict[geohash]
  except:
    df['way_type'][index] = 'NaN'
    fail_count += 1

print(fail_count)

2509


In [25]:
len(df)

100804

In [26]:
df.head()

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE,gh4,gh5,gh6,gh7,gh8,way_type
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018.0,1.0,25.0,JUEVES,10.0,NUEVO LEÓN,JUAREZ,25/01/2018,9u8c,9u8cc,9u8cck,9u8cckb,9u8cckbn,"[residential, residential, residential, reside..."
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018.0,1.0,19.0,VIERNES,9.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,19/01/2018,9u8d,9u8dq,9u8dqc,9u8dqcb,9u8dqcb4,"[service, service, service, service, service, ..."
6,D00007,25.660916,-100.282504,64820,Avenida Revolución,Valle del Huajuco,COLISION Y/O VUELCO,Auto,PLATA,2006,NaN,NaN,2018.0,1.0,22.0,LUNES,12.0,NUEVO LEÓN,MONTERREY,22/01/2018,9u89,9u89y,9u89yp,9u89yp2,9u89yp2t,"[primary, residential, residential, tertiary, ..."
7,D00008,25.723590,-100.198780,67117,Boulevard Acapulco,REAL DE SAN MIGUEL,COLISION Y/O VUELCO,Auto,BLANCO,2000,Sin daño,Frontal,2018.0,1.0,29.0,LUNES,17.0,NUEVO LEÓN,GUADALUPE,29/01/2018,9u8d,9u8dr,9u8drf,9u8drfv,9u8drfv6,"[secondary, secondary, secondary, secondary, s..."
8,D00009,25.708889,-100.246876,66480,PUERTO MARQUEZ 805,NUEVO MUNDO,COLISION Y/O VUELCO,Auto,AMARILLO,2011,Bajo,Frontal,2018.0,2.0,26.0,LUNES,2.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,26/02/2018,9u8d,9u8dq,9u8dqb,9u8dqb4,9u8dqb4m,"[secondary, secondary, secondary]"


In [27]:
road_dict = {}
for index, row in df.iterrows():
  for road in row['way_type']:
    if road in road_dict:
      road_dict[road] += 1
    else:
      road_dict[road] = 1


In [28]:
print(road_dict)

{'residential': 313991, 'service': 361878, 'footway': 139453, 'primary': 79531, 'tertiary': 160610, 'tertiary_link': 46650, 'primary_link': 66091, 'secondary': 130054, 'secondary_link': 56432, 'trunk': 77956, 'pedestrian': 4378, 'trunk_link': 134053, 'N': 5018, 'a': 2509, 'steps': 13887, 'unclassified': 19343, 'cycleway': 1040, 'track': 1173, 'motorway': 2957, 'motorway_link': 8665, 'living_street': 3446, 'path': 5711, 'construction': 332, 'bus_guideway': 192, 'bridleway': 414, 'bus_stop': 214, 'rest_area': 116, 'escape': 6, 'proposed': 8, 'raceway': 180, 'services': 34}


In [29]:
rank = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary', 'residential', 'service']

road_dict = {}

for index, row in df.iterrows():
  for road in row['way_type']:
    if rank[0] in road:
      df['way_type'][index] = [rank[0]]
      break
    elif rank[1] in road:
      df['way_type'][index] = [rank[1]]
      break
    elif rank[2] in road:
      df['way_type'][index] = [rank[2]]
      break
    elif rank[3] in road:
      df['way_type'][index] = [rank[3]]
      break
    elif rank[4] in road:
      df['way_type'][index] = [rank[4]]
      break
    elif rank[5] in road:
      df['way_type'][index] = [rank[5]]
      break
    elif rank[6] in road:
      df['way_type'][index] = [rank[6]]
      break
    
  for road in row['way_type']:
    if road in road_dict:
      road_dict[road] += 1
    else:
      road_dict[road] = 1

In [30]:
print(road_dict)

{'residential': 313991, 'service': 361878, 'footway': 139453, 'primary': 79531, 'tertiary': 160610, 'tertiary_link': 46650, 'primary_link': 66091, 'secondary': 130054, 'secondary_link': 56432, 'trunk': 77956, 'pedestrian': 4378, 'trunk_link': 134053, 'N': 5018, 'a': 2509, 'steps': 13887, 'unclassified': 19343, 'cycleway': 1040, 'track': 1173, 'motorway': 2957, 'motorway_link': 8665, 'living_street': 3446, 'path': 5711, 'construction': 332, 'bus_guideway': 192, 'bridleway': 414, 'bus_stop': 214, 'rest_area': 116, 'escape': 6, 'proposed': 8, 'raceway': 180, 'services': 34}


In [31]:
df.head(n = 20)

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE,gh4,gh5,gh6,gh7,gh8,way_type
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018.0,1.0,25.0,JUEVES,10.0,NUEVO LEÓN,JUAREZ,25/01/2018,9u8c,9u8cc,9u8cck,9u8cckb,9u8cckbn,[residential]
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018.0,1.0,19.0,VIERNES,9.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,19/01/2018,9u8d,9u8dq,9u8dqc,9u8dqcb,9u8dqcb4,[service]
6,D00007,25.660916,-100.282504,64820,Avenida Revolución,Valle del Huajuco,COLISION Y/O VUELCO,Auto,PLATA,2006,NaN,NaN,2018.0,1.0,22.0,LUNES,12.0,NUEVO LEÓN,MONTERREY,22/01/2018,9u89,9u89y,9u89yp,9u89yp2,9u89yp2t,[primary]
7,D00008,25.723590,-100.198780,67117,Boulevard Acapulco,REAL DE SAN MIGUEL,COLISION Y/O VUELCO,Auto,BLANCO,2000,Sin daño,Frontal,2018.0,1.0,29.0,LUNES,17.0,NUEVO LEÓN,GUADALUPE,29/01/2018,9u8d,9u8dr,9u8drf,9u8drfv,9u8drfv6,[secondary]
8,D00009,25.708889,-100.246876,66480,PUERTO MARQUEZ 805,NUEVO MUNDO,COLISION Y/O VUELCO,Auto,AMARILLO,2011,Bajo,Frontal,2018.0,2.0,26.0,LUNES,2.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,26/02/2018,9u8d,9u8dq,9u8dqb,9u8dqb4,9u8dqb4m,[secondary]
9,D00010,25.684047,-100.213976,67160,Avenida Plutarco Elias Calles,La Joya INFONAVIT 2do. Sector,COLISION Y/O VUELCO,Auto,AZUL,2011,Sin daño,Trasero,2018.0,1.0,1.0,LUNES,0.0,NUEVO LEÓN,GUADALUPE,1/1/2018,9u8d,9u8dp,9u8dpe,9u8dped,9u8dpedk,[secondary]
10,D00011,25.724544,-100.373691,64619,ABEDUL,VILLA DORADA,COLISION Y/O VUELCO,Auto,ROJO,2015,Bajo,Costado der central,2018.0,1.0,1.0,LUNES,0.0,NUEVO LEÓN,MONTERREY,1/1/2018,9u8d,9u8d7,9u8d7g,9u8d7gn,9u8d7gn0,[tertiary]
11,D00012,25.631471,-100.300988,64760,ALFONSO REYES,15 de Septiembre,COLISION Y/O VUELCO,Auto,BLANCO,2000,Sin daño,Costado izq delantero,2018.0,1.0,1.0,LUNES,2.0,NUEVO LEÓN,MONTERREY,1/1/2018,9u89,9u89v,9u89vd,9u89vd5,9u89vd54,[secondary]
12,D00013,25.652481,-100.208989,67169,SAN SEBASTIAN,Los Faisanes Sector el Dorado,COLISION Y/O VUELCO,Auto,PLATA,2017,NaN,NaN,2018.0,1.0,1.0,LUNES,4.0,NUEVO LEÓN,GUADALUPE,1/1/2018,9u89,9u89z,9u89zt,9u89zty,9u89ztyh,[residential]
13,D00014,25.647176,-100.180161,67193,Avenida Eloy Cavazos,Jardines de Andalucía,COLISION Y/O VUELCO,Auto,NEGRO,2009,Medio,Trasero,2018.0,1.0,1.0,LUNES,4.0,NUEVO LEÓN,GUADALUPE,1/1/2018,9u8c,9u8cb,9u8cbk,9u8cbkg,9u8cbkgj,[residential]


In [32]:
road_dict = {}
for index, row in df.iterrows():
  for road in row['way_type']:
    if road in road_dict:
      road_dict[road] += 1
    else:
      road_dict[road] = 1

print(road_dict)

{'residential': 33527, 'service': 6677, 'primary': 11694, 'secondary': 15064, 'tertiary': 15218, 'trunk': 14434, 'N': 5018, 'a': 2509, 'unclassified': 1755, 'footway': 2437, 'motorway': 932, 'track': 94, 'path': 29, 'living_street': 150, 'pedestrian': 78, 'steps': 27, 'escape': 2, 'construction': 32, 'bridleway': 2}


In [45]:
# por si la vuelvo a regar
df1 = df.copy()

to_erase = ['unclassified', 'footway', 'track', 'path', 'living_street', 'pedestrian', 'steps', 'escape', 'construction', 'bridleway']

for index, row in df1.iterrows():
  for road in row['way_type']:
    if to_erase[0] in road or to_erase[1] in road or to_erase[2] in road or to_erase[3] in road or to_erase[4] in road or to_erase[5] in road or to_erase[6] in road or to_erase[7] in road or to_erase[8] in road or to_erase[9] in road:
      df['way_type'][index] = 'NaN'
    break

In [46]:
road_dict = {}
for index, row in df1.iterrows():
  for road in row['way_type']:
    if road in road_dict:
      road_dict[road] += 1
    else:
      road_dict[road] = 1

print(road_dict)

{'residential': 33527, 'service': 6677, 'primary': 11694, 'secondary': 15064, 'tertiary': 15218, 'trunk': 14434, 'N': 6516, 'a': 3258, 'motorway': 932}


In [47]:
df1.head()

,FOLIO_ID,LAT,LONG,CODIGO POSTAL,CALLE,COLONIA,CAUSA SINIESTRO,TIPO VEHICULO,COLOR,MODELO_VEHICULO,NIVEL DAÑO VEHICULO,PUNTO DE IMPACTO,AÑO,MES,DÍA NUMERO,DIA,HORA,ESTADO,CIUDAD_APROXIMADA,DATE,gh4,gh5,gh6,gh7,gh8,way_type
1,D00002,25.647351,-100.140132,67277,AMERICA 65,BALCONES DE ZINDARO,COLISION Y/O VUELCO,Camión,BLANCO,2009,Sin daño,Costado izq trasero,2018.0,1.0,25.0,JUEVES,10.0,NUEVO LEÓN,JUAREZ,25/01/2018,9u8c,9u8cc,9u8cck,9u8cckb,9u8cckbn,[residential]
3,D00004,25.718027,-100.249990,66480,Avenida Conductores,Conductores Infonavit,COLISION Y/O VUELCO,Auto,GRIS,2016,Sin daño,Frontal,2018.0,1.0,19.0,VIERNES,9.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,19/01/2018,9u8d,9u8dq,9u8dqc,9u8dqcb,9u8dqcb4,[service]
6,D00007,25.660916,-100.282504,64820,Avenida Revolución,Valle del Huajuco,COLISION Y/O VUELCO,Auto,PLATA,2006,NaN,NaN,2018.0,1.0,22.0,LUNES,12.0,NUEVO LEÓN,MONTERREY,22/01/2018,9u89,9u89y,9u89yp,9u89yp2,9u89yp2t,[primary]
7,D00008,25.723590,-100.198780,67117,Boulevard Acapulco,REAL DE SAN MIGUEL,COLISION Y/O VUELCO,Auto,BLANCO,2000,Sin daño,Frontal,2018.0,1.0,29.0,LUNES,17.0,NUEVO LEÓN,GUADALUPE,29/01/2018,9u8d,9u8dr,9u8drf,9u8drfv,9u8drfv6,[secondary]
8,D00009,25.708889,-100.246876,66480,PUERTO MARQUEZ 805,NUEVO MUNDO,COLISION Y/O VUELCO,Auto,AMARILLO,2011,Bajo,Frontal,2018.0,2.0,26.0,LUNES,2.0,NUEVO LEÓN,SAN NICOLAS DE LOS GARZA,26/02/2018,9u8d,9u8dq,9u8dqb,9u8dqb4,9u8dqb4m,[secondary]


In [48]:
road_dict = {}
for index, row in df1.iterrows():
  for road in row['way_type']:
    if road in road_dict:
      road_dict[road] += 1
    else:
      road_dict[road] = 1

print(road_dict)

{'residential': 33527, 'service': 6677, 'primary': 11694, 'secondary': 15064, 'tertiary': 15218, 'trunk': 14434, 'N': 6516, 'a': 3258, 'motorway': 932}


In [49]:
PATH_TO_SAVE = "gdrive/My Drive/DATLAS/ChoquesYSiniestros_2020_hashedWithType.csv'"

df.to_csv(PATH_TO_SAVE)